![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2Flegacy&file=Vertex+AI+GenAI+For+BigQuery+Metadata+-+Make+Better+Tables.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/legacy/Vertex%20AI%20GenAI%20For%20BigQuery%20Metadata%20-%20Make%20Better%20Tables.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2Flegacy%2FVertex%2520AI%2520GenAI%2520For%2520BigQuery%2520Metadata%2520-%2520Make%2520Better%2520Tables.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/legacy/Vertex%20AI%20GenAI%20For%20BigQuery%20Metadata%20-%20Make%20Better%20Tables.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/legacy/Vertex%20AI%20GenAI%20For%20BigQuery%20Metadata%20-%20Make%20Better%20Tables.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

---

**File Move Notices**

This file moved locations:
- On 09/07/2024 (mm/dd/yyyy)
	- From: `Applied GenAI/Vertex AI GenAI For BigQuery Metadata - Make Better Tables.ipynb`
	- To: `Applied GenAI/legacy/Vertex AI GenAI For BigQuery Metadata - Make Better Tables.ipynb`
---
<!---end of move notices--->

# Vertex AI GenAI For BigQuery Metadata - Make Better Tables

BigQuery tables are a great source of information for generative AI applications.  Retrieving information is a multi-step process as covered in [Vertex AI GenAI For BigQuery Q&A - Overview](./Vertex%20AI%20GenAI%20For%20BigQuery%20Q%26A%20-%20Overview.ipynb).  The ability of a large language model to understand the contents of tables directly relies on the descriptiveness of the metadata: column names, column descriptions, table names, table descriptions.  

This workflow shows the potential for creating better, more descriptive metadata for BigQuery tables by using the existing metadata as well as common values from the tables columns.

This notebooks uses the BigFrames API for BigQuery to make local work in the form of a Pandas like API while keeping the execution remote, within BigQuery.  The LLM used here is Vertex AI [text-bison](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text) called directly from BigQuery using [ML.GENERATE_TEXT](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text) using the api [bigframes.ml.llm.PaLM2TextGenerator()](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.llm.PaLM2TextGenerator).


---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Vertex%20AI%20GenAI%20For%20BigQuery%20Metadata%20-%20Make%20Better%20Tables.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment.  Also, the API for Artifact Registry needs to be enabled (if not already enabled).

### Installs (If Needed)
The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name)
packages = [
    ('bigframes', 'bigframes'),
    ('google.cloud.bigquery_connection_v1', 'google-cloud-bigquery-connection')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user

### API Enablement

Make sure the [BigQuery Connection API](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) is enabled:

In [5]:
!gcloud services enable bigqueryconnection.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'bq-metadata'

In [3]:
# make this the BQ Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = EXPERIMENT
BQ_REGION = REGION[0:2] # subset to first two characters for multi-region

In [4]:
from IPython.display import Markdown
import bigframes.pandas as bf
import bigframes.ml as bfml
from bigframes.ml import llm
from bigframes.ml import ensemble
from google.cloud import bigquery_connection_v1 as bq_connection

In [5]:
bf.reset_session()
bf.options.bigquery.project = BQ_PROJECT
bf.options.bigquery.location = BQ_REGION
bf_session = bf.get_global_session()

---
## Review Data Source

The data source here is a product catalog with source:
- BigQuery Public table `bigquery-public-data.thelook_ecommerce.products`.



In [6]:
BQ_SOURCE = 'bigquery-public-data.thelook_ecommerce.products'

### Get Table: BigQuery Public Table

In [7]:
products = bf.read_gbq(BQ_SOURCE)

HTML(value='Query job 6adddf89-088a-45fe-bdaf-d21597e172be is RUNNING. <a target="_blank" href="https://consol…

In [8]:
products.dtypes

id                                  Int64
cost                              Float64
category                  string[pyarrow]
name                      string[pyarrow]
brand                     string[pyarrow]
retail_price                      Float64
department                string[pyarrow]
sku                       string[pyarrow]
distribution_center_id              Int64
dtype: object

In [9]:
products.describe()

HTML(value='Query job b9295f80-b7f9-402e-9f62-84b90fcc00b0 is DONE. 0 Bytes processed. <a target="_blank" href…

HTML(value='Query job 2c383bd2-514f-44be-9015-1128298a1175 is DONE. 931.8 kB processed. <a target="_blank" hre…

HTML(value='Query job b5e5c29c-9905-42b4-a722-d70259635665 is DONE. 0 Bytes processed. <a target="_blank" href…

,id,cost,retail_price,distribution_center_id
count,29120.0,29120.0,29120.0,29120.0
mean,14560.5,28.481774,59.220164,4.982898
std,8406.364256,30.624681,65.888927,2.901153
min,1.0,0.0083,0.02,1.0
25%,7241.0,11.13539,24.0,2.0
50%,14416.0,19.728,39.990002,5.0
75%,21787.0,34.37,69.860001,8.0
max,29120.0,557.151002,999.0,10.0


In [10]:
products.head()

HTML(value='Query job 551d9a4e-71a0-4e42-988a-b7eeefb47605 is DONE. 233.0 kB processed. <a target="_blank" hre…

HTML(value='Query job b763e074-5047-4064-9123-3f5283c487a8 is DONE. 4.5 MB processed. <a target="_blank" href=…

HTML(value='Query job 47ec6d7d-c4e9-4462-abbf-6540393b149f is DONE. 0 Bytes processed. <a target="_blank" href…

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,27569,92.652563,Swim,2XU Men's Swimmers Compression Long Sleeve Top,2XU,150.410004,Men,B23C5765E165D83AA924FA8F13C05F25,1
1,27445,24.719661,Swim,TYR Sport Men's Square Leg Short Swim Suit,TYR,38.990002,Men,2AB7D3B23574C3DEA2BD278AFD0939AB,1
2,27457,15.8976,Swim,TYR Sport Men's Solid Durafast Jammer Swim Suit,TYR,27.6,Men,8F831227B0EB6C6D09A0555531365933,1
3,27466,17.85,Swim,TYR Sport Men's Swim Short/Resistance Short Sw...,TYR,30.0,Men,67317D6DCC4CB778AEB9219565F5456B,1
4,27481,29.408001,Swim,TYR Alliance Team Splice Jammer,TYR,45.950001,Men,213C888198806EF1A0E2BBF2F4855C6C,1


### Get Table Info From BigQuery Information Schema: Columns

Retrieve the metadata for the table from Information Schema views like [INFORMATION_SCHEMA.COLUMN_FIELD_PATHS](https://cloud.google.com/bigquery/docs/information-schema-column-field-paths)

**NOTE** When `column_name` is not equal to `field_path` it is because the column is nested withing a RECORD (think array, or list) or STRUCT (think dictionary of key:value pairs).  This example does not have examples of these but could be extended to handle these as well.

In [11]:
products_columns = bf.read_gbq(f"""
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE TABLE_NAME = 'products'
        AND column_name = field_path
""")

HTML(value='Query job 28354512-aed0-4555-9fb0-2857272267df is DONE. 10.5 MB processed. <a target="_blank" href…

HTML(value='Query job 2408a003-a0a5-4902-94f4-2000039430e4 is RUNNING. <a target="_blank" href="https://consol…

In [12]:
products_columns

HTML(value='Query job d2b9a9c8-f28c-4da3-90ba-acb43981bc1d is DONE. 0 Bytes processed. <a target="_blank" href…

HTML(value='Query job 8c2897a4-aac3-49dd-9948-abc453fa78fb is DONE. 833 Bytes processed. <a target="_blank" hr…

HTML(value='Query job 4443fad2-e105-488a-b56a-1d39634ced9d is DONE. 0 Bytes processed. <a target="_blank" href…

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,bigquery-public-data,thelook_ecommerce,products,id,id,INT64,<NA>,NULL,<NA>
1,bigquery-public-data,thelook_ecommerce,products,cost,cost,FLOAT64,<NA>,NULL,<NA>
2,bigquery-public-data,thelook_ecommerce,products,category,category,STRING,<NA>,NULL,<NA>
3,bigquery-public-data,thelook_ecommerce,products,name,name,STRING,<NA>,NULL,<NA>
4,bigquery-public-data,thelook_ecommerce,products,brand,brand,STRING,<NA>,NULL,<NA>
5,bigquery-public-data,thelook_ecommerce,products,retail_price,retail_price,FLOAT64,<NA>,NULL,<NA>
6,bigquery-public-data,thelook_ecommerce,products,department,department,STRING,<NA>,NULL,<NA>
7,bigquery-public-data,thelook_ecommerce,products,sku,sku,STRING,<NA>,NULL,<NA>
8,bigquery-public-data,thelook_ecommerce,products,distribution_center_id,distribution_center_id,INT64,<NA>,NULL,<NA>


### Get Table Info From BigQuery Information Schema: Table

Retrieve the metadata for the table from Information Schema views like [INFORMATION_SCHEMA.TABLE_OPTIONS](https://cloud.google.com/bigquery/docs/information-schema-table-options)

This view has one row for each option within each table.  Here, only the `OPTION_NAME = 'description'` is needed.

In [13]:
products_table = bf.read_gbq(f"""
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.INFORMATION_SCHEMA.TABLE_OPTIONS`
    WHERE TABLE_NAME = 'products'
        AND OPTION_NAME = 'description'
""")

HTML(value='Query job b7897ace-446d-41e6-8b72-8daddc550255 is DONE. 10.5 MB processed. <a target="_blank" href…

HTML(value='Query job 9244578b-fef9-49a0-b430-6240225bd067 is RUNNING. <a target="_blank" href="https://consol…

In [14]:
products_table

HTML(value='Query job f9bfa309-ee67-43b3-af2d-307f9f3c1545 is DONE. 0 Bytes processed. <a target="_blank" href…

HTML(value='Query job 2ab580ab-89fa-4948-8322-0e720a6e2a2d is DONE. 139 Bytes processed. <a target="_blank" hr…

HTML(value='Query job 331792da-8182-46e6-9af3-0df9a3e86f0f is DONE. 0 Bytes processed. <a target="_blank" href…

,table_catalog,table_schema,table_name,option_name,option_type,option_value
0,bigquery-public-data,thelook_ecommerce,products,description,STRING,"""The Look fictitious e-commerce dataset - prod..."


### Get Values From Columns: Most common values as examples

Retrieve a sample of common values from each column to use as examples for an LLM to create names and descriptions.

Create syntax for query that will create a row per column with a sample of values from the column.

In [15]:
for c, col in enumerate(products_columns.column_name.unique().tolist()):
    if c == 0: 
        cte = f"""SELECT '{col}' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT({col}, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))"""
    else:
        cte += f"""\nUNION ALL\nSELECT '{col}' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT({col}, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))"""
print(cte)

HTML(value='Query job ceec049f-9194-4c1a-b234-7b025eab7e52 is DONE. 160 Bytes processed. <a target="_blank" hr…

SELECT 'id' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(id, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))
UNION ALL
SELECT 'cost' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(cost, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))
UNION ALL
SELECT 'category' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(category, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))
UNION ALL
SELECT 'name' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(name, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))
UNION ALL
SELECT 'brand' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(brand, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.produc

In [16]:
products_sample = bf.read_gbq(cte)

HTML(value='Query job 2c86f5c0-4f6b-4477-8ceb-0dc71fb85d05 is DONE. 4.3 MB processed. <a target="_blank" href=…

HTML(value='Query job 0e2373a6-b6cb-444f-8ff4-2054978f48ed is RUNNING. <a target="_blank" href="https://consol…

In [17]:
products_sample

HTML(value='Query job 6c625635-7ae4-4eac-86cf-ad275a3ab49a is DONE. 0 Bytes processed. <a target="_blank" href…

HTML(value='Query job 1bbda9fd-94f3-4719-a5d4-cb19f131142c is DONE. 1.5 kB processed. <a target="_blank" href=…

HTML(value='Query job e91d2839-946a-40b9-918b-96ece1d5b0e7 is DONE. 0 Bytes processed. <a target="_blank" href…

,column_name,column_sample
0,id,"29120,29119,29118,29117,29116,29115,29114,2911..."
1,name,Wrangler Men's Premium Performance Cowboy Cut ...
2,sku,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06..."
3,department,"Women,Men"
4,distribution_center_id,"2,1,3,8,4,9,7,6,5,10"
5,brand,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N..."
6,category,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ..."
7,retail_price,"25,29.989999771118164,19.989999771118164,39.99..."
8,cost,"13.549999985843897,10.750000039115548,12.05000..."


---
## BigQuery ML: Connect To Vertex AI LLMs with ML.GENERATE_TEXT

BigQuery ML can `Create Model`s that are actually connections to Remote Models. [Reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model)

Using the `REMOTE_SERVICE_TYPE = "CLOUD_AI_LARGE_LANGUAGE_MODEL_V1"` option will link to LLMs in Vertex AI!

### Connection Requirement

To make a remote connection using BigQuery ML, BigQuery uses a CLOUD_RESOURCE connection. [Reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model#connection)

Create a new connection with type `CLOUD_RESOURCE`: First, check for existing connection.

In [18]:
try:
    response = bq_connection.ConnectionServiceClient().get_connection(
            request = bq_connection.GetConnectionRequest(
                name = f"projects/{BQ_PROJECT}/locations/{BQ_REGION}/connections/{SERIES}_{EXPERIMENT}"
            )
    )
    print(f'Found existing connection with service account: {response.cloud_resource.service_account_id}')
    service_account = response.cloud_resource.service_account_id
except Exception:
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": f"projects/{BQ_PROJECT}/locations/{BQ_REGION}",
            "connection_id": f"{SERIES}_{EXPERIMENT}",
            "connection": bq_connection.types.Connection(
                {
                    "friendly_name": f"{SERIES}_{EXPERIMENT}",
                    "cloud_resource": bq_connection.CloudResourceProperties({})
                }
            )
        }
    )
    response = bq_connection.ConnectionServiceClient().create_connection(request)
    print(f'Created new connection with service account: {response.cloud_resource.service_account_id}')
    service_account = response.cloud_resource.service_account_id
    # assign the service account the Vertex AI User Role:
    !gcloud projects add-iam-policy-binding {BQ_PROJECT} --member=serviceAccount:{service_account} --role=roles/aiplatform.user

Found existing connection with service account: bqcx-1026793852137-tqpc@gcp-sa-bigquery-condel.iam.gserviceaccount.com


**NOTE**: The step above created a service account and assigned it the Vertex AI User Role.  This may take a moment to be recognized in the steps below.  If you get an error in one of the cells below try rerunning it.

### Create The Remote Model In BigQuery

Create a temp model that connects to text generation model on Vertex AI - [Reference](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.llm.PaLM2TextGenerator)

In [19]:
textgen_model = bfml.llm.PaLM2TextGenerator(
    session = bf_session,
    connection_name = f'{BQ_PROJECT}.{BQ_REGION}.{SERIES}_{EXPERIMENT}'
)

HTML(value='Query job 1e657b4e-f216-4da9-b35a-bf36bfd84485 is RUNNING. <a target="_blank" href="https://consol…

---
## Generate Table Metadata

### Bring Together Column Information

In [20]:
products_columns.columns

Index(['table_catalog', 'table_schema', 'table_name', 'column_name',
       'field_path', 'data_type', 'description', 'collation_name',
       'rounding_mode'],
      dtype='object')

In [21]:
products_sample.columns

Index(['column_name', 'column_sample'], dtype='object')

In [22]:
products_columns = products_columns[['column_name', 'data_type', 'description']].merge(products_sample, on = 'column_name')

In [23]:
products_columns

HTML(value='Query job 5c2d750c-69f1-42d6-8470-efd67a4a4fbf is DONE. 176 Bytes processed. <a target="_blank" hr…

HTML(value='Query job 22e822ad-4c3b-4523-a44e-726f2d2493d6 is DONE. 1.7 kB processed. <a target="_blank" href=…

HTML(value='Query job 3f0e6495-e3a7-47b9-b247-360e042a386e is DONE. 0 Bytes processed. <a target="_blank" href…

,column_name,data_type,description,column_sample
0,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911..."
1,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000..."
2,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ..."
3,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...
4,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N..."
5,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99..."
6,department,STRING,<NA>,"Women,Men"
7,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06..."
8,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10"


### Add Table Information

In [24]:
products_columns['table_name'] = products_table['table_name'].iloc[0]
products_columns['table_description'] = products_table['option_value'].iloc[0]

HTML(value='Query job 1e5f49c7-bf9f-4593-b8ef-56c11a76be50 is DONE. 18 Bytes processed. <a target="_blank" hre…

HTML(value='Query job 2618f695-6534-471b-aee7-6bd3483f1b19 is DONE. 67 Bytes processed. <a target="_blank" hre…

In [25]:
products_columns

HTML(value='Query job e2c0a65e-1fbf-459c-b82b-dd29df5d8885 is DONE. 176 Bytes processed. <a target="_blank" hr…

HTML(value='Query job 8774747a-02de-4edd-a502-346b3c18ed9a is DONE. 1.7 kB processed. <a target="_blank" href=…

HTML(value='Query job 6939cf6e-bed9-4aa9-a244-b72511457053 is DONE. 0 Bytes processed. <a target="_blank" href…

,column_name,data_type,description,column_sample,table_name,table_description
0,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911...",products,"""The Look fictitious e-commerce dataset - prod..."
1,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000...",products,"""The Look fictitious e-commerce dataset - prod..."
2,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ...",products,"""The Look fictitious e-commerce dataset - prod..."
3,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...,products,"""The Look fictitious e-commerce dataset - prod..."
4,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N...",products,"""The Look fictitious e-commerce dataset - prod..."
5,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99...",products,"""The Look fictitious e-commerce dataset - prod..."
6,department,STRING,<NA>,"Women,Men",products,"""The Look fictitious e-commerce dataset - prod..."
7,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06...",products,"""The Look fictitious e-commerce dataset - prod..."
8,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10",products,"""The Look fictitious e-commerce dataset - prod..."


### Create Column Naming Prompt

In [26]:
products_columns['name_prompt'] = (
    'Generate a new column name for a BigQuery column with the following information. '
    + 'The current column name is ' + products_columns['column_name'] + '. '
    + 'The table has the name ' + products_columns['table_name'] + '. '
    + 'The column has a datatype of ' + products_columns['data_type'] + ' with common values like: ' + products_columns['column_sample'] + '.'
)

In [27]:
products_columns

HTML(value='Query job dd19c5a0-5f41-4ad3-a849-132694831f66 is DONE. 176 Bytes processed. <a target="_blank" hr…

HTML(value='Query job b9af6cd0-45d8-4b67-a382-09beba6ac447 is DONE. 1.7 kB processed. <a target="_blank" href=…

HTML(value='Query job 4cdbc5e5-ce48-4415-81d6-bfe7d386f0d7 is DONE. 0 Bytes processed. <a target="_blank" href…

,column_name,data_type,description,column_sample,table_name,table_description,name_prompt
0,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
1,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
2,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
3,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...,products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
4,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
5,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
6,department,STRING,<NA>,"Women,Men",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
7,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
8,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...


In [28]:
products_columns['name_prompt'].iloc[2]

HTML(value='Query job 35a74be3-8c91-4d6c-a639-7ab648178cc9 is DONE. 1.7 kB processed. <a target="_blank" href=…

'Generate a new column name for a BigQuery column with the following information. The current column name is category. The table has the name products. The column has a datatype of STRING with common values like: Intimates,Jeans,Tops & Tees,Fashion Hoodies & Sweatshirts,Swim,Sleep & Lounge,Shorts,Sweaters,Accessories,Active.'

### Generate New Column Names

In [29]:
products_columns = products_columns.join(textgen_model.predict(products_columns['name_prompt']).rename(columns={'ml_generate_text_llm_result':'new_column_name'}))

HTML(value='Query job 4d06026c-3978-4318-ba46-92239bc6381f is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job e0cf6524-1e4d-4f7a-ba90-dc363e27a8f0 is DONE. 72 Bytes processed. <a target="_blank" hre…

In [30]:
products_columns

HTML(value='Query job 76366042-4154-42a7-8d5c-065039fa35dd is DONE. 392 Bytes processed. <a target="_blank" hr…

HTML(value='Query job a5715370-7bd3-4c69-abeb-6ebbd15e65a9 is DONE. 2.0 kB processed. <a target="_blank" href=…

HTML(value='Query job 50b772c0-c027-4331-805b-a841d2bfb597 is DONE. 0 Bytes processed. <a target="_blank" href…

,column_name,data_type,description,column_sample,table_name,table_description,name_prompt,new_column_name
0,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_id
1,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_cost
2,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_category_v2
3,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...,products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_name
4,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_brand
5,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_retail_price_float64
6,department,STRING,<NA>,"Women,Men",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_department_gender
7,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_sku
8,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_distribution_center_id


### Generate New Column Description

In [32]:
#products_columns = products_columns.drop(columns = 'new_column_description')

In [33]:
products_columns = products_columns.join(textgen_model.predict(
    'The context for a BigQuery table column follows. '
    + 'The column name is ' + products_columns['new_column_name'] + '. '
    + 'The table has the name ' + products_columns['table_name'] + '. '
    + 'The column has a datatype of ' + products_columns['data_type'] + ' with common values like: ' + products_columns['column_sample'] + '. '
    + 'Generate a description the column.'
).rename(columns={'ml_generate_text_llm_result':'new_column_description'}))

HTML(value='Query job 1954bc42-5a73-4674-8ef4-43a29bf3bfd9 is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job 317a4828-9afa-46e7-8c10-290cc12acc65 is DONE. 72 Bytes processed. <a target="_blank" hre…

In [34]:
products_columns

HTML(value='Query job c2a148ad-98e1-44ad-81a8-57916c79a6d8 is DONE. 464 Bytes processed. <a target="_blank" hr…

HTML(value='Query job db99071a-4263-4c6b-9332-83b61cdfe4e3 is DONE. 4.4 kB processed. <a target="_blank" href=…

HTML(value='Query job 3430f43b-dbca-4cac-8501-5af0b0ed357b is DONE. 0 Bytes processed. <a target="_blank" href…

,column_name,data_type,description,column_sample,table_name,table_description,name_prompt,new_column_name,new_column_description
0,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_id,product_id is a column in the products table....
1,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_cost,product_cost is a column in the products tabl...
2,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_category_v2,products_category_v2 column in the products t...
3,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...,products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_name,product_name: The name of the product. Exampl...
4,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_brand,product_brand column in the products table co...
5,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_retail_price_float64,The products_retail_price_float64 column in t...
6,department,STRING,<NA>,"Women,Men",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_department_gender,The products_department_gender column in the ...
7,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_sku,product_sku is a column in the products table...
8,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_distribution_center_id,The products_distribution_center_id column in...


### Generate New Table Description

Convert selected column for schema into a markdown table for including in the prompt:

In [37]:
markdown_products_columns = products_columns[['new_column_name', 'new_column_description', 'data_type']].rename(columns = {'new_column_name':'column_name', 'new_column_description':'description'}).to_pandas().to_markdown(index = False)

HTML(value='Query job 8502404c-ff26-47b8-91d6-91fbf6665d1d is DONE. 0 Bytes processed. <a target="_blank" href…

Review the markdown table:

In [38]:
Markdown(markdown_products_columns)

| column_name                     | description                                                                                                                                                                                                                                                                                                                                                                                                                                                              | data_type   |
|:--------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------|
| product_id                      | product_id is a column in the products table. The column is of type INT64 and has values like 29120,29119,29118,29117,29116,29115,29114,29113,29112,29111. The column likely represents the unique identifier for each product in the table.                                                                                                                                                                                                                             | INT64       |
| product_cost                    | product_cost is a column in the products table. The column contains FLOAT64 values representing the cost of each product. Example values include 13.549999985843897, 10.750000039115548, and 12.05000001937151.                                                                                                                                                                                                                                                          | FLOAT64     |
| products_category_v2            | products_category_v2 column in the products table contains product category information. The column is of type STRING and contains values such as Intimates, Jeans, Tops & Tees, Fashion Hoodies & Sweatshirts, Swim, Sleep & Lounge, Shorts, Sweaters, Accessories, and Active.                                                                                                                                                                                         | STRING      |
| product_name                    | product_name: The name of the product. Examples: Wrangler Men's Premium Performance Cowboy Cut Jean,Wrangler Men's Rugged Wear Classic Fit Jean,True Religion Men's Ricky Straight Jean,Thorlo Unisex Experia Running Sock,Puma Men's Socks,Pearl iZUMi Attack Sock 3-Pack,Fruit of the Loom Women's 6-Pack Crew Socks,7 For All Mankind Men's Standard Classic Straight Leg Jean,Wrangler Men's Wrancher Dress Jean,Wrangler Men's Original Cowboy Cut Relaxed Fit Jean | STRING      |
| product_brand                   | product_brand column in the products table contains STRING data and captures the brand name of the product. Example values include: Allegra K, Calvin Klein, Carhartt, Hanes, Volcom, Nautica, Levi's, Quiksilver, Tommy Hilfiger, Columbia.                                                                                                                                                                                                                             | STRING      |
| products_retail_price_float64   | The products_retail_price_float64 column in the products table contains the retail price of each product in US dollars. The column has a datatype of FLOAT64 and contains values like 25, 29.99, 19.99, 39.99, 49.99, 24.99, 34.99, 49.5, 50, and 55.                                                                                                                                                                                                                    | FLOAT64     |
| products_department_gender      | The products_department_gender column in the products table stores the department and gender that a product belongs to. The column has a datatype of STRING and common values include "Women" and "Men". This column can be used to filter products by department or gender, or to create reports on product sales by department or gender.                                                                                                                              | STRING      |
| product_sku                     | product_sku is a column in the products table. It is a string column with values that are product SKUs.                                                                                                                                                                                                                                                                                                                                                                  | STRING      |
| products_distribution_center_id | The products_distribution_center_id column in the products table is an INT64 data type. The column contains the ID of the distribution center where the product is stored. The most common values for this column are 2, 1, 3, 8, 4, 9, 7, 6, 5, and 10.                                                                                                                                                                                                                 | INT64       |

Generate the table description:

In [39]:
table_description = textgen_model.predict(bf.DataFrame({ "prompt": [
f"""Generate a description for the BigQuery table with schema:
{markdown_products_columns}
"""
],})).ml_generate_text_llm_result.iloc[0]

Markdown(table_description)

HTML(value='Load job 938cc49d-175c-41ee-a922-064f420f128c is RUNNING. <a target="_blank" href="https://console…

HTML(value='Query job 6579a3c4-6816-45c4-8fe3-19762cc8e68b is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job 9875befd-9b58-4f84-9851-d6e9a3c1f7d6 is DONE. 8 Bytes processed. <a target="_blank" href…

HTML(value='Query job b6316d92-431a-44d1-abd8-f0ffbf603cd5 is DONE. 285 Bytes processed. <a target="_blank" hr…

 The products table contains information about products, including product id, cost, category, name, brand, retail price, department and gender, SKU, and distribution center ID. The table can be used to analyze product sales, track inventory, and manage product distribution.

---
## Update Metadata For A Table

To show how to do an inplace update of column names and descriptions for tables and columns, the source table will be copied to a new dataset first.


**NOTE: SWITCHING TO BIGQUERY API**

Some of the tasks in this section require using feature of BigQuery DDL to alter tables that are not yet available in the BigFrames API.

### Setup BigQuery Client

In [40]:
from google.cloud import bigquery
# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

### Create BigQuery Dataset

In [41]:
# create/link to dataset
ds = bigquery.DatasetReference(BQ_PROJECT, BQ_DATASET)
ds.location = BQ_REGION
ds.labels = {'series': f'{SERIES}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True) 

### Copy Source Table To User Dataset

In [42]:
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
SELECT *
FROM `{BQ_SOURCE}`
"""
print(query)


CREATE OR REPLACE TABLE `statmike-mlops-349915.applied_genai.bq-metadata` AS
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.products`



In [43]:
job = bq.query(query = query)
job.result()
job.state

'DONE'

### Review Schema Before Update

In [44]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE TABLE_NAME = '{BQ_TABLE}'
        AND column_name = field_path
"""
bq.query(query = query).to_dataframe()

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,statmike-mlops-349915,applied_genai,bq-metadata,id,id,INT64,None,NULL,None
1,statmike-mlops-349915,applied_genai,bq-metadata,cost,cost,FLOAT64,None,NULL,None
2,statmike-mlops-349915,applied_genai,bq-metadata,category,category,STRING,None,NULL,None
3,statmike-mlops-349915,applied_genai,bq-metadata,name,name,STRING,None,NULL,None
4,statmike-mlops-349915,applied_genai,bq-metadata,brand,brand,STRING,None,NULL,None
5,statmike-mlops-349915,applied_genai,bq-metadata,retail_price,retail_price,FLOAT64,None,NULL,None
6,statmike-mlops-349915,applied_genai,bq-metadata,department,department,STRING,None,NULL,None
7,statmike-mlops-349915,applied_genai,bq-metadata,sku,sku,STRING,None,NULL,None
8,statmike-mlops-349915,applied_genai,bq-metadata,distribution_center_id,distribution_center_id,INT64,None,NULL,None


In [45]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.TABLE_OPTIONS`
    WHERE TABLE_NAME = '{BQ_TABLE}'
        AND OPTION_NAME = 'description'
"""
bq.query(query = query).to_dataframe()

,table_catalog,table_schema,table_name,option_name,option_type,option_value


### Change Column Names

Construct DDL to Alter Table:

In [59]:
for r, row in enumerate(products_columns[['column_name', 'new_column_name', 'new_column_description']].values.tolist()):
    if r == 0:
        query = f"ALTER TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`"
        query += f"""\n\tRENAME COLUMN {row[0]} TO {row[1]}"""
    else:
        query += f""",\n\tRENAME COLUMN {row[0]} TO {row[1]}"""
print(query)

HTML(value='Query job 5c6eaaa3-097d-4e96-97f0-c0f9ffcee87a is DONE. 0 Bytes processed. <a target="_blank" href…

ALTER TABLE `statmike-mlops-349915.applied_genai.bq-metadata`
	RENAME COLUMN id TO  product_id,
	RENAME COLUMN cost TO  product_cost,
	RENAME COLUMN category TO  products_category_v2,
	RENAME COLUMN name TO  product_name,
	RENAME COLUMN brand TO  product_brand,
	RENAME COLUMN retail_price TO  products_retail_price_float64,
	RENAME COLUMN department TO  products_department_gender,
	RENAME COLUMN sku TO  product_sku,
	RENAME COLUMN distribution_center_id TO  products_distribution_center_id


In [60]:
job = bq.query(query = query)
job.result()
job.state

'DONE'

### Change Column Descriptions

Construct DDL to Alter Table:

In [61]:
for r, row in enumerate(products_columns[['column_name', 'new_column_name', 'new_column_description']].values.tolist()):
    if r == 0:
        query = f"ALTER TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`"
        query += f"""\n\tALTER COLUMN {row[1]} SET OPTIONS (description = "{row[2].replace('"', "'")}")"""
    else:
        query += f""",\n\tALTER COLUMN {row[1]} SET OPTIONS (description = "{row[2].replace('"', "'")}")"""
print(query)

HTML(value='Query job 603e15d9-18ce-4db1-8469-78b794713f69 is DONE. 0 Bytes processed. <a target="_blank" href…

ALTER TABLE `statmike-mlops-349915.applied_genai.bq-metadata`
	ALTER COLUMN  product_id SET OPTIONS (description = " product_id is a column in the products table. The column is of type INT64 and has values like 29120,29119,29118,29117,29116,29115,29114,29113,29112,29111. The column likely represents the unique identifier for each product in the table."),
	ALTER COLUMN  product_cost SET OPTIONS (description = " product_cost is a column in the products table. The column contains FLOAT64 values representing the cost of each product. Example values include 13.549999985843897, 10.750000039115548, and 12.05000001937151."),
	ALTER COLUMN  products_category_v2 SET OPTIONS (description = " products_category_v2 column in the products table contains product category information. The column is of type STRING and contains values such as Intimates, Jeans, Tops & Tees, Fashion Hoodies & Sweatshirts, Swim, Sleep & Lounge, Shorts, Sweaters, Accessories, and Active."),
	ALTER COLUMN  product_name SET OP

In [62]:
job = bq.query(query = query)
job.result()
job.state

'DONE'

### Change Table Description

In [65]:
query = f"""ALTER TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
\tSET OPTIONS(description = "{table_description.replace('"', "'")}")"""
print(query)

ALTER TABLE `statmike-mlops-349915.applied_genai.bq-metadata`
	SET OPTIONS(description = " The products table contains information about products, including product id, cost, category, name, brand, retail price, department and gender, SKU, and distribution center ID. The table can be used to analyze product sales, track inventory, and manage product distribution.")


In [66]:
job = bq.query(query = query)
job.result()
job.state

'DONE'

### Review Schema After Update

In [67]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE TABLE_NAME = '{BQ_TABLE}'
        AND column_name = field_path
"""
bq.query(query = query).to_dataframe()

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,statmike-mlops-349915,applied_genai,bq-metadata,product_id,product_id,INT64,product_id is a column in the products table....,NULL,None
1,statmike-mlops-349915,applied_genai,bq-metadata,product_cost,product_cost,FLOAT64,product_cost is a column in the products tabl...,NULL,None
2,statmike-mlops-349915,applied_genai,bq-metadata,products_category_v2,products_category_v2,STRING,products_category_v2 column in the products t...,NULL,None
3,statmike-mlops-349915,applied_genai,bq-metadata,product_name,product_name,STRING,product_name: The name of the product. Exampl...,NULL,None
4,statmike-mlops-349915,applied_genai,bq-metadata,product_brand,product_brand,STRING,product_brand column in the products table co...,NULL,None
5,statmike-mlops-349915,applied_genai,bq-metadata,products_retail_price_float64,products_retail_price_float64,FLOAT64,The products_retail_price_float64 column in t...,NULL,None
6,statmike-mlops-349915,applied_genai,bq-metadata,products_department_gender,products_department_gender,STRING,The products_department_gender column in the ...,NULL,None
7,statmike-mlops-349915,applied_genai,bq-metadata,product_sku,product_sku,STRING,product_sku is a column in the products table...,NULL,None
8,statmike-mlops-349915,applied_genai,bq-metadata,products_distribution_center_id,products_distribution_center_id,INT64,The products_distribution_center_id column in...,NULL,None


In [69]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.TABLE_OPTIONS`
    WHERE TABLE_NAME = '{BQ_TABLE}'
        AND OPTION_NAME = 'description'
"""
bq.query(query = query).to_dataframe()['option_value'].iloc[0]

'" The products table contains information about products, including product id, cost, category, name, brand, retail price, department and gender, SKU, and distribution center ID. The table can be used to analyze product sales, track inventory, and manage product distribution."'